In [2]:
import pandas as pd
import numpy as np

# Read running data from parquet files
running_data = pd.read_parquet(r"data\run_ww_2019_d.parquet")
running_data = pd.concat(
    [running_data, pd.read_parquet(r"data\run_ww_2020_d.parquet")], axis=0)

# Remove entries that have 0km distance
running_data = running_data.drop(
    running_data[running_data["distance"] == 0].index)


# Add the week number for each run
# running_data["Week"] = running_data['datetime'].dt.isocalendar().week
running_data["Week"] = np.where(running_data["datetime"].dt.isocalendar(
).year == 2019, running_data['datetime'].dt.isocalendar().week, running_data['datetime'].dt.isocalendar().week + 53)

# Identify a marathon distance run
running_data["Marathon"] = (running_data["distance"] > 41.8) & (
    running_data["distance"] < 42.5)

# running_data = running_data[((running_data["Marathon"] == True).size > 1) & (running_data["Week"] > 16)]
running_data = running_data.sort_values(
    ["athlete", "datetime"]).reset_index(drop=True)


In [3]:
# Find runners that have completed a marathon distance run
marathon_runners = pd.unique(running_data[running_data["Marathon"]]["athlete"])
marathon_runner_details = {}
for runner in marathon_runners:
    marathon_runner_details[runner] = {"index": running_data[running_data["athlete"] == runner].index,
                                       "marathon_weeks": running_data[(running_data["athlete"] == runner) &
                                                                      (running_data["Marathon"])]["Week"]
                                       }


In [4]:
# Group data for marathon runners by up to preceeding 16 weeks before marathon date
marathon_running_data = pd.DataFrame()
for runner, details in marathon_runner_details.items():
    print(f"Runner # {runner}")
    for i, marathon_week in enumerate(details.get("marathon_weeks")):
        temp = pd.DataFrame()
        temp = running_data[(running_data["athlete"] == runner) & (running_data["Week"]
                                                                   <= marathon_week) &
                                                                   (running_data["Week"] > (marathon_week-16))].copy()
        temp.loc[:, "athlete"] = temp.athlete.map(str) + "-" + str(i)
        marathon_running_data = pd.concat([marathon_running_data, temp])


Runner # 4
Runner # 5
Runner # 6
Runner # 15
Runner # 22
Runner # 35
Runner # 44
Runner # 46
Runner # 54
Runner # 60
Runner # 61
Runner # 68
Runner # 71
Runner # 77
Runner # 81
Runner # 82
Runner # 84
Runner # 85
Runner # 87
Runner # 91
Runner # 103
Runner # 108
Runner # 113
Runner # 115
Runner # 122
Runner # 126
Runner # 136
Runner # 137
Runner # 138
Runner # 139
Runner # 141
Runner # 144
Runner # 145
Runner # 147
Runner # 151
Runner # 152
Runner # 154
Runner # 168
Runner # 170
Runner # 175
Runner # 179
Runner # 185
Runner # 210
Runner # 219
Runner # 222
Runner # 223
Runner # 230
Runner # 232
Runner # 245
Runner # 257
Runner # 259
Runner # 264
Runner # 275
Runner # 281
Runner # 283
Runner # 284
Runner # 288
Runner # 289
Runner # 292
Runner # 293
Runner # 298
Runner # 304
Runner # 309
Runner # 315
Runner # 319
Runner # 327
Runner # 329
Runner # 340
Runner # 346
Runner # 354
Runner # 355
Runner # 359
Runner # 366
Runner # 376
Runner # 377
Runner # 379
Runner # 385
Runner # 389
Runner # 

In [5]:
# Add pace data minutes per km
marathon_running_data["pace"] = marathon_running_data["duration"] / \
    marathon_running_data["distance"]
marathon_running_data


,datetime,athlete,distance,duration,gender,age_group,country,major,Week,Marathon,pace
443,2019-03-23,4-0,9.46,47.616667,M,35 - 54,United States,BOSTON 2017,12,False,5.033474
444,2019-04-02,4-0,6.45,34.250000,M,35 - 54,United States,BOSTON 2017,14,False,5.310078
445,2019-04-03,4-0,6.62,35.250000,M,35 - 54,United States,BOSTON 2017,14,False,5.324773
446,2019-04-05,4-0,6.49,33.316667,M,35 - 54,United States,BOSTON 2017,14,False,5.133539
447,2019-04-18,4-0,8.34,42.716667,M,35 - 54,United States,BOSTON 2017,16,False,5.121902
...,...,...,...,...,...,...,...,...,...,...,...
4302628,2019-07-08,37597-0,30.26,165.666667,F,18 - 34,United States,BOSTON 2015,28,False,5.474774
4302629,2019-07-18,37597-0,20.06,103.866667,F,18 - 34,United States,BOSTON 2015,29,False,5.177800
4302630,2019-07-20,37597-0,18.00,95.000000,F,18 - 34,United States,BOSTON 2015,29,False,5.277778
4302631,2019-07-24,37597-0,4.49,25.183333,F,18 - 34,United States,BOSTON 2015,30,False,5.608760


In [6]:
# Capture marathon run
labels = marathon_running_data[marathon_running_data["Marathon"]]

# Remove marathon data point
marathon_training_data = marathon_running_data[marathon_running_data["Marathon"] == False]


In [7]:
marathon_training_data_grouped = marathon_training_data.groupby(["athlete", "Week"], as_index=False).agg(
    {"distance": ["min", "max", "mean", "sum"],
     "duration": ["min", "max", "mean"],
     "pace": ["min", "max", "mean"],
     "datetime": ["count"]
     })
marathon_training_data_grouped


athlete Week distance                             duration  \
                          min    max       mean    sum         min   
0       10006-0    1    10.52  13.31  12.097500  48.39   57.950000   
1       10006-0    2     8.90  22.77  13.703333  82.22   45.000000   
2       10006-0    3     7.25  10.29   8.535000  34.14   40.300000   
3       10007-0    1     6.50  19.48  12.990000  25.98   33.800000   
4       10007-0    2    25.82  25.82  25.820000  25.82  121.900000   
...         ...  ...      ...    ...        ...    ...         ...   
145293   9998-1   75    12.28  16.55  14.415000  28.83   77.000000   
145294   9999-0   62    18.63  18.63  18.630000  18.63   95.000000   
145295   9999-0   66     5.43   5.43   5.430000   5.43   26.666667   
145296   9999-0   70     9.99   9.99   9.990000   9.99   45.666667   
145297   9999-0   75    19.73  19.73  19.730000  19.73   93.783333   

                                    pace                     datetime  
               max        mean       min       max      mean    count  
0        70.000000   64.333333  5.259204  5.508555  5.325658        4  
1       112.000000   69.566667  4.795991  5.352887  5.106964        6  
2        57.800000   45.704167  4.965863  5.617104  5.345638        4  
3        92.000000   62.900000  4.722793  5.200000  4.961396        2  
4       121.900000  121.900000  4.721146  4.721146  4.721146        1  
...            ...         ...       ...       ...       ...      ...  
145293  111.000000   94.000000  6.270358  6.706949  6.488653        2  
145294   95.000000   95.000000  5.099302  5.099302  5.099302        1  
145295   26.666667   26.666667  4.910988  4.910988  4.910988        1  
145296   45.666667   45.666667  4.571238  4.571238  4.571238        1  
145297   93.783333   93.783333  4.753337  4.753337  4.753337        1  

[145298 rows x 13 columns]

In [8]:
# Find the maximum number of training weeks between marathons.

max_week = {runner: week for runner, week in zip(
    marathon_training_data_grouped["athlete"], marathon_training_data_grouped["Week"])}
max_week


{'10006-0': 3,
 '10007-0': 16,
 '10010-0': 16,
 '10010-1': 56,
 '10011-0': 25,
 '10012-0': 16,
 '10012-1': 28,
 '10015-0': 16,
 '10015-1': 22,
 '10015-2': 45,
 '10015-3': 90,
 '10017-0': 70,
 '10020-0': 16,
 '10023-0': 7,
 '10024-0': 26,
 '10025-0': 15,
 '10026-0': 105,
 '10028-0': 44,
 '10029-0': 43,
 '10029-1': 47,
 '1003-0': 43,
 '10031-0': 14,
 '10040-0': 16,
 '10041-0': 78,
 '10043-0': 15,
 '10043-1': 45,
 '10043-2': 73,
 '10043-3': 74,
 '10045-0': 27,
 '10048-0': 15,
 '10057-0': 15,
 '10061-0': 90,
 '10062-0': 16,
 '10071-0': 17,
 '10071-1': 41,
 '10071-2': 98,
 '10073-0': 37,
 '10080-0': 39,
 '10084-0': 42,
 '10086-0': 16,
 '10086-1': 90,
 '10087-0': 6,
 '10089-0': 6,
 '10089-1': 36,
 '10089-2': 70,
 '10089-3': 95,
 '10090-0': 93,
 '10092-0': 63,
 '10108-0': 75,
 '10108-1': 81,
 '10113-0': 13,
 '10122-0': 14,
 '10132-0': 14,
 '10132-1': 20,
 '10137-0': 78,
 '10137-1': 95,
 '10138-0': 77,
 '10148-0': 66,
 '10153-0': 17,
 '10153-1': 19,
 '10153-2': 25,
 '10153-3': 40,
 '10153-4': 

In [9]:
marathon_training_data_grouped_adjusted = marathon_training_data_grouped.copy()
for runner, current_week in zip(marathon_training_data_grouped["athlete"], marathon_training_data_grouped["Week"]):
    marathon_training_data_grouped_adjusted.loc[(marathon_training_data_grouped_adjusted["athlete"] == runner)
                                                & (marathon_training_data_grouped_adjusted["Week"] == current_week),
                                                "adjusted_week"] = current_week - (max_week.get(runner)-16)
marathon_training_data_grouped_adjusted["adjusted_week"] = marathon_training_data_grouped_adjusted["adjusted_week"].astype(
    "int32")

marathon_training_data_grouped_adjusted


athlete Week distance                             duration  \
                          min    max       mean    sum         min   
0       10006-0    1    10.52  13.31  12.097500  48.39   57.950000   
1       10006-0    2     8.90  22.77  13.703333  82.22   45.000000   
2       10006-0    3     7.25  10.29   8.535000  34.14   40.300000   
3       10007-0    1     6.50  19.48  12.990000  25.98   33.800000   
4       10007-0    2    25.82  25.82  25.820000  25.82  121.900000   
...         ...  ...      ...    ...        ...    ...         ...   
145293   9998-1   75    12.28  16.55  14.415000  28.83   77.000000   
145294   9999-0   62    18.63  18.63  18.630000  18.63   95.000000   
145295   9999-0   66     5.43   5.43   5.430000   5.43   26.666667   
145296   9999-0   70     9.99   9.99   9.990000   9.99   45.666667   
145297   9999-0   75    19.73  19.73  19.730000  19.73   93.783333   

                                    pace                     datetime  \
               max        mean       min       max      mean    count   
0        70.000000   64.333333  5.259204  5.508555  5.325658        4   
1       112.000000   69.566667  4.795991  5.352887  5.106964        6   
2        57.800000   45.704167  4.965863  5.617104  5.345638        4   
3        92.000000   62.900000  4.722793  5.200000  4.961396        2   
4       121.900000  121.900000  4.721146  4.721146  4.721146        1   
...            ...         ...       ...       ...       ...      ...   
145293  111.000000   94.000000  6.270358  6.706949  6.488653        2   
145294   95.000000   95.000000  5.099302  5.099302  5.099302        1   
145295   26.666667   26.666667  4.910988  4.910988  4.910988        1   
145296   45.666667   45.666667  4.571238  4.571238  4.571238        1   
145297   93.783333   93.783333  4.753337  4.753337  4.753337        1   

       adjusted_week  
                      
0                 14  
1                 15  
2                 16  
3                  1  
4                  2  
...              ...  
145293            16  
145294             3  
145295             7  
145296            11  
145297            16  

[145298 rows x 14 columns]

In [43]:
df_total_distance = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "distance", "sum")).rename(columns=lambda col: f"week_{col}").add_suffix("_total_distance")

df_longest_distance = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "distance", "max")).rename(columns=lambda col: f"week_{col}").add_suffix("_longest_distance")

df_average_distance = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "distance", "mean")).rename(columns=lambda col: f"week_{col}").add_suffix("_average_distance")

df_shortest_distance = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "distance", "min")).rename(columns=lambda col: f"week_{col}").add_suffix("_shortest_distance")

df_shortest_time = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "duration", "min")).rename(columns=lambda col: f"week_{col}").add_suffix("_shortest_time")

df_longest_time = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "duration", "max")).rename(columns=lambda col: f"week_{col}").add_suffix("_longest_time")

df_average_time = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "duration", "mean")).rename(columns=lambda col: f"week_{col}").add_suffix("_average_duration")

df_fastest_pace = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "pace", "min")).rename(columns=lambda col: f"week_{col}").add_suffix("_fastest_pace")

df_slowest_pace = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "pace", "max")).rename(columns=lambda col: f"week_{col}").add_suffix("_slowest_pace")

df_average_pace = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "pace", "mean")).rename(columns=lambda col: f"week_{col}").add_suffix("_average_pace")

df_total_runs = marathon_training_data_grouped_adjusted.pivot(index="athlete", columns="adjusted_week", values=(
    "datetime", "count")).rename(columns=lambda col: f"week_{col}").add_suffix("_total_runs")

df_combined = pd.merge(
    df_total_distance, df_longest_distance, on="athlete", how="left")
df_combined = pd.merge(df_combined, df_average_distance,
                       on="athlete", how="left")
df_combined = pd.merge(df_combined, df_shortest_distance,
                       on="athlete", how="left")
df_combined = pd.merge(df_combined, df_shortest_time,
                       on="athlete", how="left")
df_combined = pd.merge(df_combined, df_longest_time, on="athlete", how="left")
df_combined = pd.merge(df_combined, df_average_time, on="athlete", how="left")
df_combined = pd.merge(df_combined, df_fastest_pace, on="athlete", how="left")
df_combined = pd.merge(df_combined, df_slowest_pace, on="athlete", how="left")
df_combined = pd.merge(df_combined, df_average_pace, on="athlete", how="left")
df_combined = pd.merge(df_combined, df_total_runs, on="athlete", how="left")
df_combined = df_combined.fillna(0)

In [44]:
df_combined = pd.merge(df_combined, marathon_training_data.loc[:, [
                       "athlete", "gender", "age_group", "country"]].drop_duplicates(), left_on=df_combined.index, right_on="athlete", how="left")
df_combined = pd.merge(df_combined, labels.loc[:, [
                       "athlete", "distance", "duration", "pace"]].drop_duplicates("athlete"), on="athlete", how="left")


In [45]:
df_combined.to_csv(r"data\marathon_runners.csv")


In [46]:
data = pd.read_csv(r"data\marathon_runners.csv", header=0)
data.head()

,Unnamed: 0,week_1_total_distance,week_2_total_distance,week_3_total_distance,week_4_total_distance,week_5_total_distance,week_6_total_distance,week_7_total_distance,week_8_total_distance,week_9_total_distance,...,week_14_total_runs,week_15_total_runs,week_16_total_runs,athlete,gender,age_group,country,distance,duration,pace
0,0,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,...,4.0,6.0,4.0,10006-0,M,35 - 54,Mexico,41.92,181.0,4.317748
1,1,25.980,25.82,31.10,50.15,59.74,6.53,22.12,45.89,61.049,...,3.0,1.0,1.0,10007-0,F,18 - 34,United States,42.43,215.0,5.067169
2,2,22.400,61.02,33.04,14.98,19.16,9.66,31.75,11.74,22.870,...,3.0,3.0,1.0,10010-0,F,35 - 54,United States,42.43,244.0,5.750648
3,3,39.359,31.07,31.31,27.39,24.41,31.74,10.15,16.96,25.490,...,3.0,3.0,2.0,10010-1,F,35 - 54,United States,42.28,251.0,5.936613
4,4,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.120,...,3.0,2.0,2.0,10011-0,F,18 - 34,United States,42.24,208.0,4.924242
